In [76]:
import numpy as np
from Bio import SeqIO

## Reading the fasta files

In the file promoters, I have 2 parts from Registry that we want to check and a third one which certainly has a RBS.

In [48]:
seqs = []
for seq_record in SeqIO.parse("Promoters.fasta", "fasta"):
    print 'Reading ' + str(seq_record.id) + "..."
    seqs.append( seq_record ) 

Reading BBa_R0010...
Reading BBa_K914003...
Reading BBa_K611012...


Just an example of a sequence and its id.

In [49]:
print seqs[0].id
print seqs[0].seq

BBa_R0010
caatacgcaaaccgcctctccccgcgcgttggccgattcattaatgcagctggcacgacaggtttcccgactggaaagcgggcagtgagcgcaacgcaattaatgtgagttagctcactcattaggcaccccaggctttacactttatgcttccggctcgtatgttgtgtggaattgtgagcggataacaatttcacaca


We now read all possible RBSs.

In [50]:
SetPOssB = [29, 30, 31, 32, 33, 34, 35, 64]
SetPOssJ = range(0,40)

RBSseqs = []

for seq_record in SeqIO.parse("../Registry_AllParts.fasta", "fasta"):
    if ( str(seq_record.id)[:-2] == "BBa_B00" and int( str(seq_record.id)[-2:] ) in SetPOssB ): 
        RBSseqs.append( seq_record )
    elif ( str(seq_record.id)[:-2] == "BBa_J611" and int( str(seq_record.id)[-2:] ) in SetPOssJ ) :
        RBSseqs.append( seq_record )
    elif ( seq_record.id == "BBa_J01010" or seq_record.id == "BBa_J01080" ):
        RBSseqs.append( seq_record )

## Strict comparison

Comparing with part placI

In [51]:
idx = []
for j in range(len(RBSseqs)):
    if RBSseqs[j].seq in seqs[0].seq : idx.append(j)

print idx

[]


Same thing with prham

In [52]:
idx = []
for j in range(len(RBSseqs)):
    if RBSseqs[j].seq in seqs[1].seq : idx.append(j)

print idx

[]


Finally, our sanity check:

In [53]:
idx = []
for j in range(len(RBSseqs)):
    if RBSseqs[j].seq in seqs[2].seq : idx.append(j)

print idx

[5]


In [54]:
for j in idx: print RBSseqs[j].id

BBa_B0034


Now, this only means none of those RBSs are included in any of the two first DNA. However, a similar gene may be encoded there. Let's try alinging them.

## Comparing alignments

In [55]:
one_sequence = 'actgatcgattgatcgatcgatcg'
another_sequence   = 'tttagatcgatctttgatc'
 
# here are the five bits of information we described before
def score_match(subject, query, subject_start, query_start, length):
    score = 0
    # for each base in the match
    for i in range(0,length):
        # first figure out the matching base from both sequences
        subject_base = subject[subject_start + i]
        query_base = query[query_start + i]
        # then adjust the score up or down depending on 
        # whether or not they are the same
        if subject_base == query_base:
            score = score + 1
        else:
            score = score - 1
    return score
 
# here is the score for the match we were looking at above
print(score_match(one_sequence, another_sequence, 7, 4, 8))
 
# let's try a few other potential matches
 
# here is the same match but shorter
print(score_match(one_sequence, another_sequence, 7, 4, 4))
 
# how about a longer match
print(score_match(one_sequence, another_sequence, 7, 4, 12))
 
# and a random match
print(score_match(one_sequence, another_sequence, 10, 1, 5))

6
2
4
-1


In [81]:
# the arguments are the five bits of information that define a match
def pretty_print_match(subject, query, subject_start, query_start, length):
    
    seqcmp = ""
    
    # first print the start/stop positions for the subject sequence
    seqcmp += str(subject_start) + (' ' * length) + str(subject_start+length) + '\n'
 
    # then print the bit of the subject that matches
    seqcmp += ' ' + subject[subject_start:subject_start+length] + '\n'
 
    # then print the bit of the query that matches
    seqcmp += ' ' + query[query_start:query_start+length] + '\n'
 
    # finally print the start/stop positions for the query
    seqcmp += str(query_start) + (' ' * length) + str(query_start+length) + '\n'
 
    seqcmp += 'n--------------------n'
    
    return seqcmp

In [96]:
def try_all_matches(subject, query, score_limit):
    setMatches = []
    setMatches_seqs = []
    
    qlen = len(query)
    slen = len(subject)
    for subject_start in range(0,slen):
        for query_start in range(0,qlen+1):
            for length in range(qlen-5,qlen+1):
                if (subject_start + length < slen and query_start + length < qlen):
                    score = score_match(subject, query, subject_start, query_start, length)
                    # only print a line of output if the score is better than some limie
                    if (score >= score_limit):
                        setMatches.append( float(score)/qlen )
                        setMatches_seqs.append( pretty_print_match(subject, query, subject_start, query_start, length) )
    
    return np.array( setMatches ), setMatches_seqs

In [97]:
setMatches, setMatches_seqs = try_all_matches(seqs[2].seq, RBSseqs[5].seq, 10)

for idx in np.argwhere(setMatches == np.amax(setMatches)):
    print "Score: ", setMatches[idx]
    print setMatches_seqs[idx]

Score:  [ 0.91666667]
0           11
 aaagaggagaa
 aaagaggagaa
0           11
n--------------------n
Score:  [ 0.91666667]
1021           1032
 aaagaggagaa
 aaagaggagaa
0           11
n--------------------n


The whole RBS seems compatible around 1021... Which totally seems reasonable.

In [98]:
setMatches, setMatches_seqs = try_all_matches(seqs[2].seq, RBSseqs[4].seq, 7)

for idx in np.argwhere(setMatches == np.amax(setMatches)):
    print "Score: ", setMatches[idx]
    print setMatches_seqs[idx]

Score:  [ 0.63636364]
456       463
 tcacaca
 tcacaca
0       7
n--------------------n
Score:  [ 0.63636364]
834         843
 tcacactgg
 tcacacagg
0         9
n--------------------n


In [108]:
setMatches, setMatches_seqs = try_all_matches(RBSseqs[5].seq, RBSseqs[4].seq, 7)

setMatches.shape

(0,)

In [110]:
for j in range(len(RBSseqs)):
    setMatches, setMatches_seqs = try_all_matches(seqs[0].seq, RBSseqs[j].seq, 8)
    if setMatches.shape[0] > 0:
        print 'RBS index: ' + j
        for idx in np.argwhere(setMatches == np.amax(setMatches)):
            print "Score: ", setMatches[idx]
            print setMatches_seqs[idx]

In [111]:
for j in range(len(RBSseqs)):
    setMatches, setMatches_seqs = try_all_matches(seqs[1].seq, RBSseqs[j].seq, 8)
    if setMatches.shape[0] > 0:
        print 'RBS index: ' + j
        for idx in np.argwhere(setMatches == np.amax(setMatches)):
            print "Score: ", setMatches[idx]
            print setMatches_seqs[idx]